In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma


In [2]:
%run /Users/enzo-macmini/auction_alg_ITU/parallel_auction_ITU.ipynb

In [666]:
n_small = 5
m_small = 3
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (2,24) , random_seed= 778, lbs =(0,0), tol = 1e-1)

In [667]:
example_small.A_ij

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [972]:
def iteration(self,i_ent, mu, V_j):
    C  = np.zeros(self.n, dtype= bool)
    C_i_ent = np.zeros(self.n, dtype= bool)
    C[i_ent] = 1
    B = np.ones(self.m, dtype= bool)
    L_j = np.ones(self.m, dtype= int) *(self.n + 1)
    d_j = np.ones(self.m, dtype= bool) * np.inf

    iter = 0
    while np.any(C == 1) and iter < 15:
        iter += 1
        i = np.argmax(C)
        print(f"iter {iter}, i: {i}")
        C[i] = 0
        C_i_ent[i] = 1
        U_ij_s = self.get_U_ij(V_j, i)[0]

        π_i = np.max(U_ij_s)
        
        suboptimal_js = U_ij_s < π_i
        print(suboptimal_js)
        d_j[suboptimal_js] = np.minimum(d_j[suboptimal_js], π_i - np.maximum(U_ij_s[suboptimal_js], self.lb_U )) 
        print(np.shape(U_ij_s))
        unissigne_optimal_j = np.where((mu.sum(0) == 0) * (π_i == U_ij_s))[0]

        if len(unissigne_optimal_j)> 0 :
            L_j[unissigne_optimal_j[0]] = i
            mu[:, L_j < self.n +1 ] = 0
            mu[L_j [L_j < self.n +1], L_j < self.n +1 ] = 1
            #print(  np.shape(self.get_U_ij(V_j, L_j [L_j < self.n +1] )))
            #print( np.shape( np.ones(((L_j < self.n +1).sum() ,1 ))))
            # print(  np.concatenate( [self.get_U_ij(V_j, L_j [L_j < self.n +1] ), 
            #                                 np.ones(((L_j < self.n +1).sum() ,1 ))], axis= 1)     ) 
            w_i_s = np.sort( np.concatenate([self.get_U_ij(V_j, L_j [L_j < self.n +1] ), 
                                            np.ones(((L_j < self.n +1).sum() ,1 ))] , axis = 1))[:,-2]
            print(w_i_s)
            V_j[L_j < self.n +1] = self.get_V_ij(L_j [L_j < self.n +1]  , L_j < self.n +1 ,  w_i_s)


            #V_j[j] = self.get_V_ij(i,j,w_i)[0]
            # print(f"j : {j}")
            print(L_j)
            print("augmentation")
            return mu, V_j


        for j in np.where(B * (~suboptimal_js) >0)[0]:
            
            #U_ij = self.get_U_ij(V_j, i, j)[0]
            if mu[:,j].sum() == 1:
                i_j = np.argmax(mu[:,j])
                B[j] = 0
                C[i_j] = 1
                
                L_j[j] = i if L_j[j] == self.n + 1 else None
                print(f"j : {j}")
                print(L_j)
                print(f"{i} <- {j} ")
                #break
            # if mu[:,j].sum() == 0:
            #     # perform augmentation
                
            #     L_j[j] = i
            #     mu[:, L_j < self.n +1 ] = 0
            #     mu[L_j [L_j < self.n +1], L_j < self.n +1 ] = 1
            #     w_i_s = np.sort(self.get_U_ij(V_j, L_j [L_j < self.n +1] ), axis = 1)[:,-2]
            #     V_j[L_j < self.n +1] = self.get_V_ij(L_j [L_j < self.n +1]  , L_j < self.n +1 ,  w_i_s)
            #     #V_j[j] = self.get_V_ij(i,j,w_i)[0]
            #     print(f"AaaAAAAAA")

            #     return mu, V_j


           

    
    return C_i_ent, B * (d_j < np.inf), d_j

OneToOneITU.iteration = iteration

In [973]:
np.ones((1,1))

array([[1.]])

In [974]:
mu_ex = np.zeros([n_small,m_small])
V_ex = np.ones(example_small.m) * example_small.lb_V
perm_unassigned = np.zeros(example_small.n, dtype= bool)

In [975]:
example_small.B_ij

array([[15,  5,  9],
       [20, 14,  2],
       [22, 12,  0],
       [ 6, 23, 21],
       [18,  6, 13]])

In [1020]:
i_ent = np.where((mu_ex.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
iter_values = example_small.iteration(i_ent, mu_ex, V_ex )

if len(iter_values) == 2:
    mu_ex, V_ex = iter_values

else:
    C_i, B, d = iter_values
    V_ex[~B] += np.min(d)
    print(B)
    print(d)
    

perm_unassigned = np.all(example_small.get_U_ij(V_ex , np.arange(example_small.n)) <= example_small.lb_U, axis = 1)
mu_ex[perm_unassigned,:] = 0

iter 1, i: 4
[ True  True False]
(3,)
j : 2
[6 6 4]
4 <- 2 
iter 2, i: 3
[ True False False]
(3,)
[9. 1.]
[6 3 4]
augmentation


In [1021]:
perm_unassigned

array([ True,  True, False, False, False])

In [1022]:
mu_ex

array([[0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [1023]:
V_ex

array([20., 14., 12.])

In [1024]:
example_small.get_U_ij(V_ex , np.arange(5))

array([[ -5.,  -9.,  -3.],
       [  0.,   0., -10.],
       [  2.,  -2., -12.],
       [-14.,   9.,   9.],
       [ -2.,  -8.,   1.]])

In [1025]:
def ITU_auction(self):
    mu_ij = np.zeros([self.n,self.m])
    V_j = np.ones(self.m) * self.lb_V
    perm_unassigned = np.zeros(self.n, dtype= bool)
    
    while perm_unassigned.sum() + mu_ij.sum() < self.n :
        i_ent = np.where((mu_ij.sum(1) == 0) * (~perm_unassigned) > 0)[0][0]
        iter_values = self.iteration(i_ent, mu_ij, V_j )

        if len(iter_values) == 2:
            mu_ij, V_j = iter_values

        else:
            C_i, B, d = iter_values
            V_j[~B] += np.min(d)
            print(B)
            print(d)
            

        perm_unassigned = np.all(self.get_U_ij(V_j , np.arange(self.n)) <= self.lb_U, axis = 1)
        mu_ij[perm_unassigned,:] = 0

    return mu_ij, V_j, perm_unassigned * 1

OneToOneITU.ITU_auction = ITU_auction

In [1035]:
n_small = 10
m_small = 30
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (2,24) , random_seed= 778, lbs =(0,0), tol = 1e-1)

In [1036]:
mu_ex ,  V_ex , perm_ = example_small.ITU_auction()

iter 1, i: 0
[ True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True]
(30,)
[23.]
[11 11 11 11 11 11 11 11 11 11  0 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11]
augmentation
iter 1, i: 1
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True]
(30,)
[19.]
[11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11  1 11 11
 11 11 11 11 11 11]
augmentation
iter 1, i: 2
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False  True  True]
(30,)
[21.]
[11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11  2 11 11]
augmentation
iter 1, i: 3
[ True  True  True  True False  True  True

In [1037]:
mu_ex

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0

In [1034]:
example_small.get_U_ij(V_ex , np.arange(10))

array([[ -5., -15., -13.],
       [  0.,  -6., -20.],
       [  2.,  -8., -22.],
       [-14.,   3.,  -1.],
       [ -2., -14.,  -9.],
       [-15.,   0., -17.],
       [ -6., -12.,   1.],
       [  0., -16., -18.],
       [ -8., -17., -14.],
       [-15., -17.,  -6.]])